In [33]:
import pandas as pd
import numpy as np
import os
import scipy
import sqlite3
from collections import defaultdict

In [4]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
cur = conn.cursor()
cur.execute("select * from loan limit 5;")
results = cur.fetchall()
# print(results)
cur.close()
conn.close()

C:\Users\andre\Documents\GitHub\loan_driving2


In [5]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
print(df)
conn.close()

       loan_amnt        id  loan_status sub_grade
0         2500.0   1077430  Charged Off        C4
1         5600.0   1071795  Charged Off        F2
2         5375.0   1071570  Charged Off        B5
3         9000.0   1064687  Charged Off        C1
4        10000.0   1069057  Charged Off        B2
5        21000.0   1039153  Charged Off        B4
6         6000.0   1069559  Charged Off        B3
7        15000.0   1069800  Charged Off        C2
8         5000.0   1069657  Charged Off        D2
9         5000.0   1069465  Charged Off        A5
10       15000.0   1069248  Charged Off        B1
11       12000.0   1069243  Charged Off        C5
12       21000.0   1069410  Charged Off        E4
13       10000.0   1069126  Charged Off        B2
14       12500.0   1069093  Charged Off        D3
15        8200.0   1068906  Charged Off        F2
16        6400.0   1060981  Charged Off        D2
17       17500.0   1068542  Charged Off        D3
18        6200.0   1068509  Charged Off        B1


In [6]:
ddict = df.to_dict()
print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
print(len(list(g)))

15000.0
45248


In [32]:
# create a training set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
# print(df.axes)
# print(df.get_values()[0][1])
# print(df.get_values())
# print(df.values[3])
print(df.values[0])

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

[5000.0 '1077501' 'Fully Paid' 'B2']


In [52]:
# find all unique values for the loan status column
l_status_dict = defaultdict(int)
for row in df.itertuples():
    # print(row[3])
    if row[3] not in l_status_dict:
        l_status_dict[row[3]] += 1

# print(l_status_dict.keys())

In [53]:
print(l_status_dict.keys())

dict_keys(['Fully Paid', 'Charged Off', 'Default', 'In Grace Period', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Issued'])
